# Galaxy Classification with CNN (Pytorch)

Data:

https://www.kaggle.com/c/galaxy-zoo-the-galaxy-challenge



References:

1. https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

2. https://pytorch.org/vision/stable/models.html

3. https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

4. https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html

5. https://cs231n.github.io/transfer-learning/



## Check GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-5d99ad53-86af-b254-cd15-29cb277ee66e)


## Import libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms, utils
import torch.optim as optim
from torch.optim import lr_scheduler

import time
import os
import zipfile

%matplotlib inline

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Unzip images

Before running the code, please upload `train_46183.zip` and `test_15395.zip` to the workspace.

We upzip these 2 files, and place all training images (46,183 images) into the folder `images_train`, and all test images (15,395 images) into the folder `images_test`.

We create the folders if they don't exist.

In [3]:
## Unzip training images
train_dir = 'images_train'
# create dir if not exist
if not os.path.exists(train_dir):
    os.makedirs(train_dir)

zip_ref = zipfile.ZipFile('train_46183.zip', 'r')
zip_ref.extractall(path=train_dir) # unzip
zip_ref.close()

In [4]:
## Unzip test images
test_dir = 'images_test'
# create dir if not exist
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

zip_ref = zipfile.ZipFile('test_15395.zip', 'r')
zip_ref.extractall(path=test_dir) # unzip
zip_ref.close()

## Import custom datasets

In [5]:
## Custom Galaxy Zoo Dataset
class GalaxyZooDataset(Dataset):
    """Galaxy Zoo Dataset"""

    def __init__(self, csv_file, images_dir, transform=None):
        """
        Args:
            csv_file (string): path to the label csv
            images_dir (string): path to the dir containing all images
            transform (callable, optional): transform to apply
        """
        self.labels_df = pd.read_csv(csv_file)
        self.images_dir = images_dir
        self.transform = transform
    
    def __len__(self):
        """
        Returns the size of the dataset
        """
        return len(self.labels_df)

    def __getitem__(self, idx):
        """
        Get the idx-th sample.
		Outputs the image (channel first) and the true label
        """
        if torch.is_tensor(idx):
            idx = idx.tolist()
        
        # galaxy ID
        galaxyid = self.labels_df.iloc[idx, 0].astype(str)
		# path of the image
        image_path = os.path.join(self.images_dir, galaxyid + '.jpg')
		# read the image
        image = Image.open(image_path)
		# apply transform (optional)
        if self.transform is not None:
            image = self.transform(image)
		# read the true label
        label = int(self.labels_df.iloc[idx, 1])

        return image, label

## Data Augmentation Transforms

In [40]:
def create_data_transforms(is_for_inception=False):
    """
    Create Pytorch data transforms for the GalaxyZoo datasets.
    Args:
        is_for_inception (bool): True for inception neural networks
    Outputs:
        train_transform: transform for the training data
        test_transform: transform for the testing data
    """
    if is_for_inception:
        input_size = 299
    else:
        input_size = 224

    train_transform = transforms.Compose([transforms.Resize((input_size, input_size)),
                                            transforms.RandomResizedCrop(input_size, scale=(0.8, 1.0), ratio=(0.999, 1.001)),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.RandomVerticalFlip(),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0, 0, 0], [255, 255, 255]),
                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    test_transform = transforms.Compose([transforms.Resize((input_size, input_size)),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0, 0, 0], [255, 255, 255]),
                                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    
    return train_transform, test_transform

## Parameters

In [35]:
NUM_OF_CLASSES = 5  # there are 5 classes in total
BATCH_SIZE = 32     # batch zize

## Training function

In [19]:
def train_model(model, num_epochs, criterion, optimizer, scheduler, print_every=1, is_for_inception=False):
    """
    Train the model
    Args:
        model: Pytorch neural model
        num_epochs: number of epochs to train
        criterion: the loss function object
        optimizer: the optimizer
        scheduler: the learning rate decay scheduler
        print_every: print the information every X epochs
        is_for_inception: True if the model is an inception model
    """
    
    for epoch in range(num_epochs):
        # time of start
        epoch_start_time = time.time()

        """
        Train
        """
        model.train()

        epoch_train_cum_loss = 0.0
        epoch_train_cum_corrects = 0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.long().to(device)

            optimizer.zero_grad()
            
            if is_for_inception:
                pred_logits, aux_outputs = model(images)
                loss = criterion(pred_logits, labels) + 0.4*criterion(aux_outputs, labels)
            else:
                pred_logits = model(images)
                loss = criterion(pred_logits, labels)

            _, pred_classes = torch.max(pred_logits.detach(), dim=1)
            pred_classes = pred_classes.long()

            epoch_train_cum_loss += loss.item() * images.size(0)
            epoch_train_cum_corrects += torch.sum(pred_classes==labels.data)

            loss.backward()
            optimizer.step()
            
        """
        Eval
        """
        model.eval()

        epoch_test_cum_loss = 0.0
        epoch_test_cum_corrects = 0

        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.long().to(device)

            with torch.no_grad():
                pred_logits = model(images)
                _, pred_classes = torch.max(pred_logits.detach(), dim=1)
                loss = criterion(pred_logits, labels)

                epoch_test_cum_loss += loss.item() * images.size(0)
                epoch_test_cum_corrects += torch.sum(pred_classes==labels.data)

        scheduler.step()

        ## Calculate metrics
        train_loss = epoch_train_cum_loss / len(data_train)
        train_acc = epoch_train_cum_corrects / len(data_train)
        test_loss = epoch_test_cum_loss / len(data_test)
        test_acc = epoch_test_cum_corrects / len(data_test)
        
        epoch_end_time = time.time()
        epoch_time_used = epoch_end_time - epoch_start_time

        ## Print metrics
        if (epoch+1) % print_every == 0:
            print("Epoch {}/{}\tTrain loss: {:.4f}\tTrain acc: {:.3f}\tTest loss: {:.4f}\tTest acc: {:.3f}\tTime: {:.0f}m {:.0f}s".format(
                epoch+1, num_epochs, train_loss, train_acc, test_loss, test_acc, epoch_time_used // 60, epoch_time_used % 60))

## ResNet18 Model

### Model architecture

**Original paper**

Deep Residual Learning for Image Recognition [(arXiv)](https://arxiv.org/abs/1512.03385)

**The last layer**

The last layer of ResNet18 model is called `fc`, with input size = `512`

We replace the last layer with a linear layer by `model.fc = nn.Linear(512, NUM_OF_CLASSES, bias=True)`

In [34]:
## Resnet18 architecture
model = models.resnet18(pretrained=True)
print(model)
# count trainable parameters
print("Number of trainable parameters: {}".format(sum(param.numel() for param in model.parameters() if param.requires_grad)))
# free the space
del model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Create transforms and dataloaders

We set `is_for_inception` to `False`

In [43]:
# create transform
train_transform, test_transform = create_data_transforms(is_for_inception=False)

# create dataset
data_train = GalaxyZooDataset('class_labels_train_46183_C5.csv', 'images_train', train_transform)
data_test = GalaxyZooDataset('class_labels_test_15395_C5.csv', 'images_test', test_transform)

# dataloader
train_loader = DataLoader(data_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(data_test, batch_size=BATCH_SIZE, shuffle=True)

# check the sizes
print("Number of training data: {} ({} batches)".format(len(data_train), len(train_loader)))
print("Number of testing data: {} ({} batches)".format(len(data_test), len(test_loader)))

Number of training data: 46183 (1444 batches)
Number of testing data: 15395 (482 batches)


### Train and fine-tune the model

In [45]:
## Download the pre-trained resnet18 model
model = models.resnet18(pretrained=True)

# freeze the weights
for param in model.parameters():
    param.requires_grad = False

# change the last fc layer
model.fc = nn.Linear(512, NUM_OF_CLASSES)
print(model.fc) # print the modified last layer

print("============")
print("Training the last layer only")
print("Number of trainable parameters: {}".format(sum(param.numel() for param in model.parameters() if param.requires_grad)))
print("============")

# move to gpu
model = model.to(device)
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# train
train_model(model, 5, criterion, optimizer, scheduler, print_every=1, is_for_inception=False)


"""
Fine tuning
"""
# unfreeze the weights of the last block
for param in model.layer4.parameters():
    param.requires_grad = True

print("============")
print("Fine Tuning the whole ResNet model")
print("Number of trainable parameters: {}".format(sum(param.numel() for param in model.parameters() if param.requires_grad)))
print("============")

# move to gpu
model = model.to(device)
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=5e-4)
# scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# train
train_model(model, 20, criterion, optimizer, scheduler, print_every=1, is_for_inception=False)

Linear(in_features=512, out_features=5, bias=True)
Training the last layer only
Number of trainable parameters: 2565
Epoch 1/5	Train loss: 0.9760	Train acc: 0.607	Test loss: 0.9265	Test acc: 0.624	Time: 6m 15s
Epoch 2/5	Train loss: 0.9462	Train acc: 0.613	Test loss: 0.9789	Test acc: 0.612	Time: 6m 15s
Epoch 3/5	Train loss: 0.9374	Train acc: 0.618	Test loss: 0.9497	Test acc: 0.629	Time: 6m 15s
Epoch 4/5	Train loss: 0.9298	Train acc: 0.617	Test loss: 0.9146	Test acc: 0.624	Time: 6m 17s
Epoch 5/5	Train loss: 0.9222	Train acc: 0.620	Test loss: 0.9305	Test acc: 0.625	Time: 6m 20s
Fine Tuning the whole ResNet model
Number of trainable parameters: 8396293
Epoch 1/20	Train loss: 0.8814	Train acc: 0.639	Test loss: 0.8229	Test acc: 0.660	Time: 6m 23s
Epoch 2/20	Train loss: 0.8334	Train acc: 0.658	Test loss: 0.8372	Test acc: 0.668	Time: 6m 23s
Epoch 3/20	Train loss: 0.8128	Train acc: 0.666	Test loss: 0.7899	Test acc: 0.672	Time: 6m 26s
Epoch 4/20	Train loss: 0.7967	Train acc: 0.675	Test loss: 0.7

### Save the model weights

In [46]:
## Save the weights
torch.save(model.state_dict(), 'resnet18_tuned.pth')

In [47]:
del model

## VGG-16-bn Model

### Model architecture

**Original paper**

Very Deep Convolutional Netrowks for Large-Scale Image Recognition [(arXiv)](https://arxiv.org/abs/1409.1556)

**The last layer**

The last layer of VGG-16 model is called `classifier[6]`, with input size = `4096`

We replace the last layer with a linear layer by `model.classifier[6] = nn.Linear(4096, NUM_OF_CLASSES, bias=True)`

In [48]:
## VGG-16 (bn) architecture
model = models.vgg16_bn(pretrained=True)
print(model)
# count trainable parameters
print("Number of trainable parameters: {}".format(sum(param.numel() for param in model.parameters() if param.requires_grad)))
# free the space
del model

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

### Create transforms and dataloaders

In [49]:
# create transform
train_transform, test_transform = create_data_transforms(is_for_inception=False)

# create dataset
data_train = GalaxyZooDataset('class_labels_train_46183_C5.csv', 'images_train', train_transform)
data_test = GalaxyZooDataset('class_labels_test_15395_C5.csv', 'images_test', test_transform)

# dataloader
train_loader = DataLoader(data_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(data_test, batch_size=BATCH_SIZE, shuffle=True)

# check the sizes
print("Number of training data: {} ({} batches)".format(len(data_train), len(train_loader)))
print("Number of testing data: {} ({} batches)".format(len(data_test), len(test_loader)))

Number of training data: 46183 (1444 batches)
Number of testing data: 15395 (482 batches)


### Train and fine-tune the model

In [50]:
## Download the pre-trained VGG16 model
model = models.vgg16_bn(pretrained=True)

# freeze the weights
for param in model.parameters():
    param.requires_grad = False

# change the last fc layer
model.classifier[6] = nn.Linear(4096, NUM_OF_CLASSES)
print(model.classifier[6]) # print the modified last layer

print("============")
print("Training the last layer only")
print("Number of trainable parameters: {}".format(sum(param.numel() for param in model.parameters() if param.requires_grad)))
print("============")

# move to gpu
model = model.to(device)
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# train
train_model(model, 5, criterion, optimizer, scheduler, print_every=1)


"""
Fine tuning
"""
# unfreeze the weights of the last block
for param in model.classifier[3].parameters():
    param.requires_grad = True

print("============")
print("Fine Tuning")
print("Number of trainable parameters: {}".format(sum(param.numel() for param in model.parameters() if param.requires_grad)))
print("============")

# move to gpu
model = model.to(device)
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=5e-4)
# scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# train
train_model(model, 20, criterion, optimizer, scheduler, print_every=1)

Linear(in_features=4096, out_features=5, bias=True)
Training the last layer only
Number of trainable parameters: 20485
Epoch 1/5	Train loss: 1.0095	Train acc: 0.592	Test loss: 0.9260	Test acc: 0.622	Time: 8m 41s
Epoch 2/5	Train loss: 1.0078	Train acc: 0.592	Test loss: 0.9219	Test acc: 0.629	Time: 8m 42s
Epoch 3/5	Train loss: 0.9995	Train acc: 0.596	Test loss: 0.9255	Test acc: 0.617	Time: 8m 42s
Epoch 4/5	Train loss: 0.9918	Train acc: 0.599	Test loss: 0.9149	Test acc: 0.628	Time: 8m 44s
Epoch 5/5	Train loss: 0.9872	Train acc: 0.600	Test loss: 0.9122	Test acc: 0.631	Time: 8m 43s
Fine Tuning
Number of trainable parameters: 16801797
Epoch 1/20	Train loss: 0.9758	Train acc: 0.611	Test loss: 0.9016	Test acc: 0.631	Time: 8m 43s
Epoch 2/20	Train loss: 0.9184	Train acc: 0.628	Test loss: 0.8754	Test acc: 0.641	Time: 8m 42s
Epoch 3/20	Train loss: 0.9070	Train acc: 0.631	Test loss: 0.8737	Test acc: 0.643	Time: 8m 42s
Epoch 4/20	Train loss: 0.8978	Train acc: 0.633	Test loss: 0.8634	Test acc: 0.647	

### Save the model weights

In [51]:
## Save the weights
torch.save(model.state_dict(), 'vgg16bn_tuned.pth')

## Inception v3 Model

### Model architecture

**Original paper**

Rethinking the Inception Architecture for Computer Vision [(arXiv)](https://arxiv.org/abs/1512.00567)

**The last layer**

The last layer of Inception v3 model is called `fc`, with input size = `2048`

We replace the last layer with a linear layer by `model.fc = nn.Linear(2048, NUM_OF_CLASSES, bias=True)`

**The auxiliary layer**

In addition to the `fc` layer, the model has a second output layer `AuxLogits.fc`, with input size = `768`

We replace the auxiliary layer with a linear layer by `model.AuxLogits.fc = nn.Linear(768, NUM_OF_CLASSES, bias=True)`

In [ ]:
## Inception v3 architecture
model = models.inception_v3(pretrained=True)
print(model)
# count trainable parameters
print("Number of trainable parameters: {}".format(sum(param.numel() for param in model.parameters() if param.requires_grad)))
# free the space
del model

### Create transforms and dataloaders

In [ ]:
# create transform
train_transform, test_transform = create_data_transforms(is_for_inception=True)

# create dataset
data_train = GalaxyZooDataset('class_labels_train_46183_C5.csv', 'images_train', train_transform)
data_test = GalaxyZooDataset('class_labels_test_15395_C5.csv', 'images_test', test_transform)

# dataloader
train_loader = DataLoader(data_train, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(data_test, batch_size=BATCH_SIZE, shuffle=True)

# check the sizes
print("Number of training data: {} ({} batches)".format(len(data_train), len(train_loader)))
print("Number of testing data: {} ({} batches)".format(len(data_test), len(test_loader)))

### Train and fine-tune the model

In [ ]:
## Download the pre-trained resnet18 model
model = models.inception_v3(pretrained=True)

# freeze the weights
for param in model.parameters():
    param.requires_grad = False

# change the last fc layer
model.fc = nn.Linear(2048, NUM_OF_CLASSES, bias=True)
print(model.fc) # print the modified last layer

# change the aux fc layer
model.AuxLogits.fc = nn.Linear(768, NUM_OF_CLASSES, bias=True)
print(model.AuxLogits.fc) # print the modified aux fc layer

print("============")
print("Training the last layer + aux layer")
print("Number of trainable parameters: {}".format(sum(param.numel() for param in model.parameters() if param.requires_grad)))
print("============")

# move to gpu
model = model.to(device)
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)
# scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# train
train_model(model, 5, criterion, optimizer, scheduler, print_every=1, is_for_inception=True)


"""
Fine tuning
"""
# unfreeze more weights
for param in model.Mixed_7c.parameters():
    param.requires_grad = True

print("============")
print("Fine Tuning")
print("Number of trainable parameters: {}".format(sum(param.numel() for param in model.parameters() if param.requires_grad)))
print("============")

# move to gpu
model = model.to(device)
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=5e-4)
# scheduler
scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
# train
train_model(model, 20, criterion, optimizer, scheduler, print_every=1, is_for_inception=False)

### Save the model weights

In [ ]:
## Save the weights
torch.save(model.state_dict(), 'inceptionv3_tuned.pth')